### Overview

Input 1 DICOM volumn --> RetinaNet --> VGG --> Grad-CAM visualization

If you got an error when importing the pydicom package:
Step 1: run the following lines:
- !pip uninstall numpy
- !pip install numpy

Step 2: enter 'y' in the prompt

Step 3: delete the 2 commands in step 1

Step 4: restart the runtime and run all


In [ ]:
#  !pip uninstall numpy
#  !pip install numpy

In [ ]:

# install dependencies not included by Colab
# use pip3 to ensure compatibility w/ Google Deep Learning Images 
!pip install -q pydicom 
!pip install -q tqdm 
!pip install -q imgaug
!pip install -q pickle5

#imports needed for dbtex function
print('---dbtex installs---')
!pip install -q pylibjpeg  #for dbtex
!pip install -q python-gdcm
!pip install -q pylibjpeg-libjpeg 
!pip install -q pylibjpeg-openjpeg
!pip install -q Pillow



---dbtex installs---


In [ ]:
import sys
import os

import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import numpy as np
from pathlib import Path
import six
import csv
import logging
import math
import statistics
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from PIL import Image 
import scipy.misc
from scipy import stats
from sklearn.model_selection import train_test_split
from datetime import datetime
from scipy.spatial import distance
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
from torchvision.models.detection import RetinaNet
from torchvision.models.detection.anchor_utils import AnchorGenerator
import torch.optim as optim

from typing import AnyStr, BinaryIO, Optional, Union
from skimage.exposure import rescale_intensity

import pydicom as dicom
import pickle5 as pickle; #generic storage of image array
import PIL
import pylibjpeg

from torchvision.transforms import ToTensor, Lambda
#show model design parameters with torchsummary
import torchsummary
from torchsummary import summary
from torch import FloatTensor
from torch import tensor

# Mount the google drive so that we can access the data stored in the drive
from google.colab import drive
drive.mount('/content/gdrive')

#
# SET LOCAL IMPORT DIRECTORY FOR .PY FILES
#
sys.path.append('/content/gdrive/Shareddrives/DBTdata/code/GENERAL_FUNCTIONS/DBT_PY_FILES')
!ls '/content/gdrive/Shareddrives/DBTdata/code/GENERAL_FUNCTIONS/DBT_PY_FILES'


from VGG16 import VGG16

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
create_augmented_data.py  randomize_patches.py	VGG_Classify.py
CustomImageDataset.py	  TestImageDataset.py
__pycache__		  VGG16.py


In [ ]:
### Enable GPU, if present
train_on_gpu = torch.cuda.is_available()
if (train_on_gpu):
    !nvidia-smi -L
    !nvidia-smi 
    dev=torch.device("cuda")
else:
    print('GPU NOT FOUND!!! USING CPU INSTEAD!!!!!')

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-c50bc105-ddd4-acee-e606-0bb784171b9e)
Mon Jun  7 18:26:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    33W / 250W |   2729MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
           

In [ ]:
'''
GET DIR LISTING OF DBT CHALLENGE DATA-------------------------------------------
'''
#It is necessary to get the view number to cross-reference against the view name
#for use with the dcm read function
def get_dirs(top_dir): 
    dir_list = os.scandir(top_dir)
    directories=[]
    patient_data = {}
    for ii in dir_list:
        #print('top dir is ',ii)
        if (ii.is_dir()):
            #patient ID directory
            patient_id = ii.name
            patient_data[patient_id]={}
            directories.append(ii.name)
            #get file listings for each directory found
            full_name = os.path.join(top_dir,ii.name)


            files = os.scandir(full_name)
            for patient_uid in files:
                #print('ptuid = ',patient_uid)
                pass

            if (patient_uid.is_dir()):
                #this will be the longer directory name
                #with study id
                #This will need to be tokenized to get 
                #the two fields out for info
                #--for now, keep the longer directory name
                patient_data[patient_id][patient_uid.name]={}

                exam_listings = os.scandir(os.path.join(full_name, 
                                                  patient_uid.name))
                for exam_number in exam_listings:
                    patient_data[patient_id][patient_uid.name][exam_number.name] =[]
                    #fill this in with exam dcm contents
                    dcm_list = os.listdir(os.path.join(full_name, 
                                                  patient_uid.name,
                                                  exam_number.name))
                    for dcm_file in dcm_list:
                        patient_data[patient_id][patient_uid.name][exam_number.name].append(dcm_file)

        else:
            print('*** found single file ',ii.name)
    return patient_data

#Parse the CSV files given with the main datasets
def get_csv_data(filename):
    fields = []
    rows = []

    # reading csv file
    with open(filename, 'r') as csvfile:
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        
        # extracting field names through first row
        fields = next(csvreader)

        # extracting each data row one by one
        for row in csvreader:
            rows.append(row)


        # lines present
        print("found rows: %d"%(csvreader.line_num))
    return fields, rows

#This function calls the directory function to retrieve patient IDs
def get_pt_data(dataset = 2):
    if (dataset == 0):
        top_dir = '/content/gdrive/Shareddrives/DBTdata/DBT_DATA/TRAINING_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT'
        csv_dir = '/content/gdrive/Shareddrives/DBTdata/DBT_DATA/TRAINING_DATA/'
    elif (dataset == 2):
        print('Processing TEST SET')
        top_dir = '/content/gdrive/Shareddrives/DBTdata/DBT_DATA/TEST_DATA/manifest-1605042674814/Breast-Cancer-Screening-DBT'
        csv_dir = '/content/gdrive/Shareddrives/DBTdata/DBT_DATA/TEST_DATA/'
    else:
        print('Missing FOLDER')

    upper_dir =top_dir

    patient_data = get_dirs(upper_dir)

    print('Finished getting patient directories')
    return patient_data, top_dir, csv_dir

#READ CSV FILES
def read_csv_files(csv_dir):
    csv_list = os.listdir(csv_dir)

    #store CSV data
    path_fields =[]
    path_rows = []
    label_fields = []
    label_rows = []
    boxes_fields = []
    boxes_rows =[]

    for csv_file in csv_list:
        filename = os.path.join(csv_dir,csv_file)
        if ('path' in csv_file): #directories and patient info
            print('found path csv')
            path_fields, path_rows = get_csv_data(filename)
        elif ('labels' in csv_file): #labels
            print('found label csv')
            label_fields, label_rows = get_csv_data(filename)        
        elif ('boxes' in csv_file): #annotation data
            print('found annotation csv')
            boxes_fields, boxes_rows = get_csv_data(filename)
            print('box shape is ',np.shape(boxes_rows))

            #get average box dimensions
            xdim =[]
            ydim = []
            for ii in boxes_rows:
                xdim.append(np.uint(ii[7]))
                ydim.append(np.uint(ii[8]))

            average_x = sum(xdim)/len(xdim)
            average_y = sum(ydim)/len(ydim)
            print('average ROI x and y dimensions are: ',average_x,average_y)

        else:
            print('Non-archive file found ',filename)

    return path_fields, path_rows, label_fields, label_rows, boxes_fields, boxes_rows


In [ ]:
#Run the patient directory search
dataset = 2 #0=TRAINING, 2=TEST
patient_data,data_dir, csv_dir = get_pt_data(dataset)

Processing TEST SET
*** found single file  license.html
Finished getting patient directories


In [ ]:
path_fields, path_rows, label_fields, label_rows, boxes_fields, boxes_rows = read_csv_files(csv_dir)

Non-archive file found  /content/gdrive/Shareddrives/DBTdata/DBT_DATA/TEST_DATA/manifest-1605042674814
found path csv
found rows: 602


In [ ]:
#The DBT data needs info from one or more CSV files to map the view number with 
#its view name (needed for dcmread)
#Get the matching columns for the relevant fields
for field_number,ii in enumerate(boxes_fields): #only training had annotations
    if ('Slice' == ii):
        slice_column = field_number
    elif('View' == ii):
        view_column = field_number

#Test data has a different format from Training
for field_number, ii in enumerate(path_fields):
    if (dataset == 0):
        if ('descriptive_path' == ii):
            view_id_column = field_number
    else:
        if ('descriptive_path' == ii):
            descriptive_path_column = field_number

    if ('View' == ii):
        path_view_column = field_number
    elif ('StudyUID' == ii):
        path_studyuid_column = field_number



#for TEST data, descriptive path links to the actual dcm. Use the path_view_column
#in conjunction with descriptive path to get the file and dcmread view name

file_list = [] #full filename
view_list = [] #paired view of full filename
for row_data in path_rows:
    if (dataset == 2): #Test format\
        fields = row_data[descriptive_path_column].split('/')
        fields = fields[1:]
        full_file_name = data_dir
        for ii in fields:
            full_file_name = os.path.join(full_file_name,ii)
        if (os.path.exists(full_file_name) != True):
            print('Failed to find file: ', full_file_name)
        else:
            file_list.append(full_file_name)
            view_list.append(row_data[path_view_column])
        #print(full_file_name)
        #print(os.path.exists( full_file_name))



In [ ]:
# Functions for loading the DICOM volume
def dcmread_image(
    fp: Union[str, "os.PathLike[AnyStr]", BinaryIO],
    view: str,
    index: Optional[np.uint] = None,
) -> np.ndarray:
    """Read pixel array from DBT DICOM file"""
    ds = dicom.dcmread(fp)
    ds.decompress(handler_name='pillow') #pylibjpeg')
    pixel_array = ds.pixel_array
    view_laterality = view[0].upper()
    image_laterality = _get_image_laterality(pixel_array[index or 0])
    if index is not None:
        pixel_array = pixel_array[index]
    if not image_laterality == view_laterality:
        pixel_array = np.flip(pixel_array, axis=(-1, -2))
        print("Laterality fixed!")
    window_center = _get_window_center(ds)
    window_width = _get_window_width(ds)
    low = (2 * window_center - window_width) / 2
    high = (2 * window_center + window_width) / 2
    pixel_array = rescale_intensity(
        pixel_array, in_range=(low, high), out_range="dtype"
    )
    return pixel_array


def _get_dicom_laterality(ds: dicom.dataset.FileDataset) -> str:
    """Unreliable - DICOM laterality is incorrect for some cases"""
    return ds[0x5200, 0x9229][0][0x0020, 0x9071][0][0x0020, 0x9072].value


def _get_image_laterality(pixel_array: np.ndarray) -> str:
    left_edge = np.sum(pixel_array[:, 0])  # sum of left edge pixels
    right_edge = np.sum(pixel_array[:, -1])  # sum of right edge pixels
    return "R" if left_edge < right_edge else "L"


def _get_window_center(ds: dicom.dataset.FileDataset) -> np.float32:
    return np.float32(ds[0x5200, 0x9229][0][0x0028, 0x9132][0][0x0028, 0x1050].value)


def _get_window_width(ds: dicom.dataset.FileDataset) -> np.float32:
    return np.float32(ds[0x5200, 0x9229][0][0x0028, 0x9132][0][0x0028, 0x1051].value)

# A helper function to do zero padding on images so that we have the same size for all images.
def padding(array, xx, yy):

  h = array.shape[0]
  w = array.shape[1]

  a = (xx - h) // 2
  aa = xx - a - h

  b = (yy - w) // 2
  bb = yy - b - w

  return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

# Store all the slices and their indices in a dictionary
def create_dataset(img_array):
  dataset = []

  # Since we won't know which slice contains potential cancer, we need to go through all slices
  for box_slice_index in range(img_array.shape[0]):
    img_slice = img_array[box_slice_index, :, :]
    # Normalize the image (min max norm)
    img_slice = (img_slice - img_slice.min()) / (img_slice.max() - img_slice.min())
    img_slice = padding(img_slice, 2457, 1996)
    # Convert into a 3 channel image
    img_slice = np.stack((img_slice,)*3, axis = 0)
    data = {}
    data['img'] = img_slice
    data['slice_index'] = box_slice_index

    dataset.append(data)

  return dataset

def run_retinanet(img_data):
  # Load the trained model weights 
  retinanet = torchvision.models.detection.retinanet_resnet50_fpn(num_classes = 2, pretrained = False, pretrained_backbone = True) 
  checkpoint = torch.load('/content/gdrive/Shareddrives/DBTdata/code/RetinaNet/model_20_epochs.pth')
  retinanet.load_state_dict(checkpoint['net'])
  retinanet.to(device = dev)
  retinanet.eval()

  num_top_boxes_to_save = 3
  save_predictions = []

  to_be_predicted = create_dataset(img_data)
  # Loop through each slice in the 3D volume
  for slice_index in range(len(to_be_predicted)):

    # Convert the image array to tensor 
    image = torch.from_numpy(to_be_predicted[slice_index]['img'].astype(np.float32))
    image = image.to(device = dev) # Put it on the GPU
    image = [image]

    # The outputs consist of 300 predicted boxes, their predicted class labels, and confidence scores.
    # They're already ranked in a decreasing order according to the confidence scores
    outputs = retinanet.forward(image)
    outputs = [{k: v.detach().cpu().numpy() for k, v in t.items()} for t in outputs] # Convert tensors to numpy

    # Save the top 3 boxes with the highest confidence score
    for prediction_index in range(num_top_boxes_to_save):
      one_prediction = {}
      one_prediction['Slice'] = to_be_predicted[slice_index]['slice_index']
      pred_box = outputs[0]['boxes'][prediction_index]
      # For the predicted box, change from (x1, y1, x2, y2) to (x, y, w, h) 
      pred_box[2] -= pred_box[0]
      pred_box[3] -= pred_box[1]
      one_prediction['pred_X'] = pred_box[0]
      one_prediction['pred_Y'] = pred_box[1]
      one_prediction['pred_width'] = pred_box[2]
      one_prediction['pred_height'] = pred_box[3]
      one_prediction['confidence'] = outputs[0]['scores'][prediction_index]
      save_predictions.append(one_prediction)
    
  df_save_predictions = pd.DataFrame(save_predictions)
  return df_save_predictions

# Some boxes are exactly the same but with a different confidence threshold. 
# That's because the confidence is for both the box prediction and the classification within the retinanet. So it's not an error. 
# This function post-processes the output dataframe, so that the same boxes won't appear twice.
def post_process_outputs(df):
  to_be_deleted = []
  for i in range(len(df) - 1):
      if df['pred_X'][i] == df['pred_X'][i + 1] and df['pred_Y'][i] == df['pred_Y'][i + 1] and df['pred_width'][i] == df['pred_width'][i + 1] and df['pred_height'][i] == df['pred_height'][i + 1]:
        to_be_deleted.append(i)
  df = df.drop(to_be_deleted)
  df = df.reset_index(drop = True)
  # Only keep the boxes that have confidence above the (mean + 1 standard deviation) confidence of all predicted boxes.
  threshold = np.mean(df['confidence']) + np.std(df['confidence'])
  boxes_with_low_confidence = np.where(df['confidence'] < threshold)[0]
  df = df.drop(boxes_with_low_confidence)
  df = df.reset_index(drop = True)
  return df

def run_VGG(img_data, df_retinanet_outputs_processed):
  sys.path.append('/content/gdrive/Shareddrives/DBTdata/code/GENERAL_FUNCTIONS/DBT_PY_FILES')
  from TestImageDataset import TestImageDataset
  from VGG_Classify import VGG_Classify

  VGG_model_dir = '/content/gdrive/Shareddrives/DBTdata/code/VGG_MODELS'
  VGG_model_number = 99

  if (VGG_model_number == 1):
      model_test = 'vgg16_best_accuracy_93_EPOCH_96_0.04582521319389343'
  elif (VGG_model_number == 2):
      model_test = 'vgg16_best_accuracy_99_EPOCH_94_0.00040225533302873373'
  else:
      model_test = 'vgg16_best_accuracy_99_EPOCH_73_0.00033' #full augmentation


  final_model = os.path.join(VGG_model_dir,model_test)

  #
  # PROCESS RETINANET BOXES
  #
  numslices, numrows, numcols = np.shape(img_data) #find limits of volume
  predictions = []
  for i in range(len(df_retinanet_outputs_processed)):
      if (df_retinanet_outputs_processed['Slice'][i] == 0): #skip slice 0, for now (VGG needs slice-1)
          print('Slice is too low')
          continue
      if (df_retinanet_outputs_processed['Slice'][i] == numslices-1): #due to 0 ordering
          print('At upper edge of volume')
          continue
      box_info = []
      box_info.append('unknownFileName')
      box_info.append(df_retinanet_outputs_processed['Slice'][i]) #slice
      box_info.append(int(df_retinanet_outputs_processed['pred_Y'][i]))#row corner
      box_info.append(int(df_retinanet_outputs_processed['pred_height'][i])) #row height
      box_info.append(int(df_retinanet_outputs_processed['pred_X'][i])) #col corner
      box_info.append(int(df_retinanet_outputs_processed['pred_width'][i])) #col width
      #Use the current box information as area for VGG
      prediction = VGG_Classify(img_data, box_info,final_model)
      predictions.append(prediction)
      print('----Predicted class number is ', prediction)
  
  # If 1 box were predicted as cancer, then this entire case is cancer (denoted as 1)
  if predictions.count(1) >= 1:
    predicted_labels_this_case = 1
  else:
    predicted_labels_this_case = 0

  return predicted_labels_this_case, predictions

#Read in files from the DBT Test directory

In [ ]:
def main(num_cases_to_predict):
  output_path = '/content/gdrive/Shareddrives/DBTdata/model_final_output/from_DBTChallenge_testing_set/'
  file_list_trucated = file_list[0:num_cases_to_predict]
  case_level_predictions = []
  for counter, fname in enumerate(file_list_trucated):
    img_data = dcmread_image(fname,view_list[counter])
    case_level_prediction = {}
    # Run the RetinaNet model and get predicted bounding boxes, predicted class label as output
    df_retinanet_outputs = run_retinanet(img_data)
    df_retinanet_outputs_processed = post_process_outputs(df_retinanet_outputs)
    print('Number of RetinaNet predicted boxes: ', len(df_retinanet_outputs_processed))
    print('Classification module...')
    predicted_labels_this_case, predicted_classes_all_boxes = run_VGG(img_data, df_retinanet_outputs_processed)

    # Save the predicted labels and their corresponding boxes in a dataframe.
    df_retinanet_outputs_processed['pred_class'] = predicted_classes_all_boxes
    df_retinanet_outputs_processed['pred_class'] = df_retinanet_outputs_processed['pred_class'].map({0: 'NonCancer', 1: 'Cancer'})
    filename = (fname.split(' ')[0]).split('/')[9:11]
    print(filename)
    df_retinanet_outputs_processed.to_csv(output_path + filename[0] + filename[1] + view_list[counter] + '.csv', index = False)
    case_level_prediction['FileName'] = filename[0] + filename[1] + view_list[counter]
    if 1 in predicted_classes_all_boxes:
      case_level_prediction['PredictedClass'] = 'Cancer'
    else:
      case_level_prediction['PredictedClass'] = 'NonCancer'
    
    case_level_predictions.append(case_level_prediction)

  df_case_level_predictions = pd.DataFrame(case_level_predictions)
  df_case_level_predictions.to_csv(output_path + 'case_level_predictions.csv', index = False)

In [ ]:
main(num_cases_to_predict = 10)

/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (10-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "


Number of RetinaNet predicted boxes:  15
Classification module...
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
['DBT-P00036', '01-01-2000-DBT-S03354-MAMMO']


/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (10-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "


Number of RetinaNet predicted boxes:  22
Classification module...
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
['DBT-P00036', '01-01-2000-DBT-S03354-MAMMO']


/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (10-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "


...

Number of RetinaNet predicted boxes:  40
Classification module...
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  1
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  1
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predic

/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (10-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "


Number of RetinaNet predicted boxes:  17
Classification module...
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
['DBT-P00092', '01-01-2000-DBT-S01654-MAMMO']


/usr/local/lib/python3.7/dist-packages/pydicom/pixel_data_handlers/pillow_handler.py:177: UserWarning: The (0028,0101) 'Bits Stored' value (10-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  f"The (0028,0101) 'Bits Stored' value ({ds.BitsStored}-bit) "


Number of RetinaNet predicted boxes:  32
Classification module...
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predicted class number is  0
----Predic